# Table of contents:

In [1]:
import pandas as pd
import numpy as np
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
request = drive_service.files().get_media(fileId = '1XQgPGGrzOU24SGcujJOZ9mPpDDfjaV4o').execute()
import io
data = pd.read_csv(io.BytesIO(request))
data.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [2]:
data[data['Cancelled'] == 1].head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
178,2008,1,3,4,NaN,700,NaN,830,WN,126,NaN,NaN,90.0,NaN,NaN,NaN,LAS,OAK,407,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
373,2008,1,3,4,NaN,1100,NaN,1215,WN,1146,NaN,NaN,75.0,NaN,NaN,NaN,LAX,OAK,337,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
399,2008,1,3,4,NaN,905,NaN,1025,WN,469,NaN,NaN,80.0,NaN,NaN,NaN,LAX,SFO,337,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
401,2008,1,3,4,NaN,1620,NaN,1740,WN,618,NaN,NaN,80.0,NaN,NaN,NaN,LAX,SFO,337,NaN,NaN,1,C,0,NaN,NaN,NaN,NaN,NaN
415,2008,1,3,4,NaN,1930,NaN,2035,WN,2528,NaN,NaN,65.0,NaN,NaN,NaN,LAX,SJC,308,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN


In [0]:
data = data.drop(columns=['CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Year'])
data = data.dropna(subset=['DepTime', 'ArrTime', 'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
                          'DepDelay', 'TaxiIn', 'TaxiOut'])
# Tạo cột mới với encode cho target column ArrDelay
data['ArrDelay_ecd'] = np.where(data['ArrDelay'] >= 30, 1, 0)
# xóa cột ArrDelay đi vì ta đã có cột ArrDelay_ecd
data = data.drop(columns=['ArrDelay'])

In [4]:
# check missing value again
missing_val_in_column = (data.isnull().sum())
print(missing_val_in_column[missing_val_in_column >0])

Series([], dtype: int64)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [0]:
data = data.drop(columns=['TailNum', 'Origin', 'Dest','DepDelay', 'Cancelled','Diverted'])

In [7]:
data.head()

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,Distance,TaxiIn,TaxiOut,ArrDelay_ecd
0,1,3,4,2003.0,1955,2211.0,2225,WN,335,128.0,150.0,116.0,810,4.0,8.0,0
1,1,3,4,754.0,735,1002.0,1000,WN,3231,128.0,145.0,113.0,810,5.0,10.0,0
2,1,3,4,628.0,620,804.0,750,WN,448,96.0,90.0,76.0,515,3.0,17.0,0
3,1,3,4,926.0,930,1054.0,1100,WN,1746,88.0,90.0,78.0,515,3.0,7.0,0
4,1,3,4,1829.0,1755,1959.0,1925,WN,3920,90.0,90.0,77.0,515,3.0,10.0,1


In [8]:
data['ArrDelay_ecd'].value_counts()

0    5927611
1     927413
Name: ArrDelay_ecd, dtype: int64

In [0]:
data_down = data.sample(frac=0.35) # lấy mẫu ngẫu nhiên xuống còn 35%
del data

In [10]:
data_down['ArrDelay_ecd'].value_counts()

0    2074388
1     324870
Name: ArrDelay_ecd, dtype: int64

Giảm giữ liệu xuống còn 35% so với dữ liệu gốc

In [11]:
data_down[data_down['CRSElapsedTime'] < 0]

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,Distance,TaxiIn,TaxiOut,ArrDelay_ecd
3995883,7,29,2,1323.0,1330,1356.0,1110,HA,165,33.0,-140.0,23.0,100,4.0,6.0,1
1358057,3,8,6,1839.0,1756,1910.0,1735,OO,5428,31.0,-21.0,23.0,77,1.0,7.0,1
3997645,7,21,1,1327.0,1330,1355.0,1109,HA,344,28.0,-141.0,17.0,102,5.0,6.0,1
3997646,7,22,2,1327.0,1330,1359.0,1109,HA,344,32.0,-141.0,21.0,102,5.0,6.0,1


Đây là giá trị bất thường chưa xử lý kĩ ở đầu. Vì:<br>
CRSElapsedTime: scheduled elapsed time of the flight, in minutes (stored as integer). Tức là lịch trình dự kiến thời gian trong khi bay nên không thể là số âm.<br>
Giải pháp: ta remove những row này đi, ở sample này chỉ có 5 rows!

In [0]:
data_down = data_down.drop(data_down[data_down['CRSElapsedTime'] < 0].index, axis=0)

In [13]:
data_down.describe()

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,Distance,TaxiIn,TaxiOut,ArrDelay_ecd
count,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06,2.399254e+06
mean,6.389204e+00,1.573422e+01,3.924761e+00,1.333645e+03,1.324890e+03,1.481372e+03,1.493757e+03,2.212830e+03,1.273130e+02,1.290627e+02,1.040055e+02,7.286622e+02,6.864919e+00,1.644267e+01,1.354029e-01
std,3.399381e+00,8.797694e+00,1.987467e+00,4.783016e+02,4.645473e+02,5.052568e+02,4.828071e+02,1.956239e+03,7.014657e+01,6.949616e+01,6.740198e+01,5.629821e+02,4.945594e+00,1.128736e+01,3.421535e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.500000e+01,2.000000e+00,0.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.000000e+00,8.000000e+00,2.000000e+00,9.280000e+02,9.250000e+02,1.107000e+03,1.115000e+03,6.190000e+02,7.700000e+01,8.000000e+01,5.600000e+01,3.280000e+02,4.000000e+00,1.000000e+01,0.000000e+00
50%,6.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.318000e+03,1.512000e+03,1.515000e+03,1.564000e+03,1.100000e+02,1.110000e+02,8.600000e+01,5.830000e+02,6.000000e+00,1.400000e+01,0.000000e+00
75%,9.000000e+00,2.300000e+01,6.000000e+00,1.728000e+03,1.715000e+03,1.909000e+03,1.905000e+03,3.494000e+03,1.570000e+02,1.590000e+02,1.320000e+02,9.540000e+02,8.000000e+00,1.900000e+01,0.000000e+00
max,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.359000e+03,9.741000e+03,1.379000e+03,6.600000e+02,1.350000e+03,4.962000e+03,2.330000e+02,4.220000e+02,1.000000e+00


Giờ thì trông ổn rồi!

In [14]:
data_down.shape

(2399254, 16)

 'UniqueCarrier' là category value nên sẽ dùng label encoder.<br>
Sau đó dữ liệu mới dùng PCA, và SelectKbes.



In [0]:
X = data_down.drop(['ArrDelay_ecd', 'Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'], axis=1)
y = data_down["ArrDelay_ecd"] # target column

In [16]:
X.shape

(2399254, 11)

### Apply PCA

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components= 7)
X = pca.fit_transform(X)

In [0]:
n= data_down[['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier']]

In [0]:
one_hot = pd.get_dummies(data=n, columns=['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'])

In [0]:
one_hot.shape

(2399255, 70)

In [0]:
X = pd.DataFrame(X)

In [0]:
X.shape

(2399255, 7)

In [0]:
X = X.reset_index()
one_hot = one_hot.reset_index()
X = X.drop('index', axis=1) # drop cột index tạo ra khi reset_index
one_hot = one_hot.drop('index', axis=1)  # drop cột index tạo ra khi reset_index

In [0]:
new_X = pd.concat([X, one_hot], axis=1, sort=False)

In [0]:
new_X.shape

(2399255, 77)

In [0]:
y.shape

(2399255,)

**Hold out**

In [0]:
x_train, x_val, y_train, y_val = train_test_split(new_X, y, test_size = 0.2, random_state = 0)

In [0]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92    414741
           1       0.49      0.44      0.46     65110

    accuracy                           0.86    479851
   macro avg       0.70      0.68      0.69    479851
weighted avg       0.86      0.86      0.86    479851



In [0]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.92      1.00      0.96    414741
           1       0.96      0.46      0.62     65110

    accuracy                           0.92    479851
   macro avg       0.94      0.73      0.79    479851
weighted avg       0.93      0.92      0.91    479851



In [0]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    414741
           1       0.95      0.95      0.95     65110

    accuracy                           0.99    479851
   macro avg       0.97      0.97      0.97    479851
weighted avg       0.99      0.99      0.99    479851



In [0]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.98      0.99      0.99    414741
           1       0.96      0.86      0.91     65110

    accuracy                           0.98    479851
   macro avg       0.97      0.93      0.95    479851
weighted avg       0.98      0.98      0.98    479851



In [0]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(x_train, y_train)
y_pred = gbk.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98    414741
           1       0.92      0.87      0.89     65110

    accuracy                           0.97    479851
   macro avg       0.95      0.93      0.94    479851
weighted avg       0.97      0.97      0.97    479851



In [0]:
# Support Vector Machines
from sklearn.svm import LinearSVC

svm = LinearSVC(class_weight='balanced')
svm.fit(x_train, y_train)
y_pred = svm.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

           0       0.91      1.00      0.95    414741
           1       0.97      0.38      0.55     65110

    accuracy                           0.91    479851
   macro avg       0.94      0.69      0.75    479851
weighted avg       0.92      0.91      0.90    479851



### Apply Select K best

In [0]:
from sklearn.feature_selection import SelectKBest, f_classif
X = data_down.drop(['ArrDelay_ecd', 'Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'], axis=1)
y = data_down["ArrDelay_ecd"] # target column

In [0]:
X = SelectKBest(f_classif, k=7).fit_transform(X, y)

In [0]:
type(X)

numpy.ndarray

In [0]:
X = pd.DataFrame(X)
n= data_down[['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier']]
one_hot = pd.get_dummies(data=n, columns=['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'])

In [0]:
one_hot.shape

(2399255, 70)

In [0]:
X = X.reset_index()
one_hot = one_hot.reset_index()
X = X.drop('index', axis=1) # drop cột index tạo ra khi reset_index
one_hot = one_hot.drop('index', axis=1)  # drop cột index tạo ra khi reset_index

In [0]:
new_X = pd.concat([X, one_hot], axis=1, sort=False)

In [0]:
new_X.shape

(2399255, 77)

**Hold out**

In [0]:
x_train, x_val, y_train, y_val = train_test_split(new_X, y, test_size = 0.2, random_state = 0)

In [0]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.74      0.82    414741
           1       0.26      0.57      0.36     65110

    accuracy                           0.72    479851
   macro avg       0.59      0.66      0.59    479851
weighted avg       0.83      0.72      0.76    479851



In [0]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.92      1.00      0.96    414741
           1       0.96      0.48      0.64     65110

    accuracy                           0.93    479851
   macro avg       0.94      0.74      0.80    479851
weighted avg       0.93      0.93      0.92    479851



In [0]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    414741
           1       0.96      0.96      0.96     65110

    accuracy                           0.99    479851
   macro avg       0.98      0.98      0.98    479851
weighted avg       0.99      0.99      0.99    479851



In [0]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.95      1.00      0.97    414741
           1       0.97      0.68      0.80     65110

    accuracy                           0.95    479851
   macro avg       0.96      0.84      0.89    479851
weighted avg       0.95      0.95      0.95    479851



In [0]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(x_train, y_train)
y_pred = gbk.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96    414741
           1       0.96      0.51      0.66     65110

    accuracy                           0.93    479851
   macro avg       0.94      0.75      0.81    479851
weighted avg       0.93      0.93      0.92    479851



In [0]:
# Support Vector Machines
from sklearn.svm import LinearSVC

linear_svc = LinearSVC(class_weight='balanced')
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

           0       0.88      1.00      0.93    414741
           1       0.97      0.10      0.18     65110

    accuracy                           0.88    479851
   macro avg       0.92      0.55      0.56    479851
weighted avg       0.89      0.88      0.83    479851



### Apply RFE

In [18]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
rfe = RFE(model, 7)
X = rfe.fit_transform(X,y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [0]:
X = pd.DataFrame(new_X)
n= data_down[['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier']]
one_hot = pd.get_dummies(data=n, columns=['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'])

In [0]:
X = X.reset_index()
one_hot = one_hot.reset_index()
X = X.drop('index', axis=1) # drop cột index tạo ra khi reset_index
one_hot = one_hot.drop('index', axis=1)  # drop cột index tạo ra khi reset_index

In [0]:
new_X = pd.concat([X, one_hot], axis=1, sort=False)

In [24]:
new_X.shape

(2399255, 77)

**Hold out**

In [0]:
x_train, x_val, y_train, y_val = train_test_split(new_X, y, test_size = 0.2, random_state = 0)

In [26]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.93      1.00      0.96    415033
           1       0.98      0.53      0.69     64818

    accuracy                           0.93    479851
   macro avg       0.95      0.76      0.82    479851
weighted avg       0.94      0.93      0.93    479851



Sau khi dùng RFE trên mô hình **Logistic Regression** thì kết quả đã được cải thiện! Cụ thể:
-  Sau khi RFE thì f1-score của **1** giờ là 0.69 cao hơn PCA (0.62), cao hơn SelectKbest (0.64)
- Ở đây khi dùng RFE, PCA, SelectKBest đều chọn ra số continous feature dữ lại là 7 và nối các feature category được encode.

**Thử thêm RFE với trường hợp mà mô hình Decision Tree. Liệu mô hình này kết quả có được cải thiện như dùng RFE với Logistic Regression?**

In [0]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
rfe = RFE(model, 7)
X = rfe.fit_transform(X,y)

In [18]:
X.shape

(2399254, 7)

In [0]:
X = pd.DataFrame(X)
n= data_down[['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier']]
one_hot = pd.get_dummies(data=n, columns=['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'])

In [0]:
X = X.reset_index()
one_hot = one_hot.reset_index()
X = X.drop('index', axis=1) # drop cột index tạo ra khi reset_index
one_hot = one_hot.drop('index', axis=1)  # drop cột index tạo ra khi reset_index

In [21]:
new_X = pd.concat([X, one_hot], axis=1, sort=False)
new_X.shape

(2399254, 77)

**Hold out**

In [0]:
x_train, x_val, y_train, y_val = train_test_split(new_X, y, test_size = 0.2, random_state = 0)

In [23]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    414684
           1       0.97      0.96      0.97     65167

    accuracy                           0.99    479851
   macro avg       0.98      0.98      0.98    479851
weighted avg       0.99      0.99      0.99    479851



Đúng như ta mong đợi, sau khi dùng RFE cho mô hình Decision Tree thì kết quả lúc này được cải thiện cao hơn. F1-score của **1** lúc này đã tăng lên đến **0.97**. Cao nhất so với khi PCA hoặc SelectKbest.

# Conclusions of this section

- Khi down sample của dữ liệu xuống còn **35%** so với dữ liệu gốc thì tốc độ tính toán và chạy mô hình nhanh hơn hẳn và kết quả dự đoán của các mô hình không thay đổi nhiều, chênh lệch tầm 1-2%.

- Khi dùng **PCA** cho bộ dữ liệu (n_components=7) thì kết quả so với dùng **SelectKbest** (k=7) ta nhận thấy ở hầu hết mô hình thì dùng PCA có kết quả tốt hơn.

- Mô hình dùng Decision Tree thì đạt hiệu quả tốt nhất.

- Khi dùng **RFE** chọn ra các continous feature cho mô hình cụ thể. Ta nhận thấy mô hình chạy tốt hơn hẳn. Cụ thể f1-score ở label **1** đều tăng.

Một số ghi chú ở phần này:

- Muốn dùng PCA hoặc SelectKbest thì chỉ được dùng cho _các feature chưa dùng_ **on hot encode**. Bởi vậy theo cách làm ở trên: ta giảm chiều trên các features không phải dạng category, dùng one hot encode trên các feature dạng category, rồi mới nối 2 cái lại với nhau.

- Trong trường hợp không dùng one hot encode, thì ta dùng label encoder các feature category rồi mới PCA hoặc SelectKbest.

Một số câu hỏi ở phần này:

- Để tìm ra mô hình tốt nhất ta dùng PCA hoặc SelectKbest , có cùng số n_components bằng số K, rồi so sánh các mô hình với nhau xem mô hình nào chạy có kết quả tốt nhất để chọn?

- Bước tiếp theo để dùng kĩ thuật Feature Selection tốt hơn. Ta dùng RFE với mô hình được cho là tốt nhất để chọn feature?